In [1]:
import sys
import re
import requests
import urllib.parse
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.distance import geodesic as GD
from geopy.geocoders import Nominatim

In [2]:
url='https://www.jobbank.gc.ca/jobsearch/jobsearch?flg=E&fcan=1&fss=1&fss=C&fwcl=B&fwcl=C&fwcl=D&fwcl=E&fglo=1&sort=M&fwht=D&fwht=M&fsrc=16&fjnc=1&fexp=0&fexp=1&fprov=AB&fprov=ON&fskl=101020&fskl=101010'
page=requests.get(url)

soup=BeautifulSoup(page.content, 'html.parser')

job_elements=soup.find_all('a', class_='resultJobItem')

In [18]:
role_list=[]
dates_list=[]
business_list=[]
locations_list=[]
salaries_list=[]
id_list=[]
noccode_list=[]
employment_type_list=[]
job_duration_list=[]
education_list=[]
experience_list=[]
address_list=[]
weekly_hours_list=[]
vacancies_list=[]
work_permit_list=[]

jobs={
    
    'ID':id_list,
    'ROLES':role_list,
    'SALARY':salaries_list,
    'EDUCATION':education_list,
    'EXPERIENCE':experience_list,
    'NOC CODE':noccode_list,
    'BUSINESS':business_list,
    'LOCATIONS':locations_list,
    'PUBLISHED DATES':dates_list,
    'EMPLOYMENT TYPE':employment_type_list,
    'JOB DURATION':job_duration_list,
    'WEEKLY HOURS':weekly_hours_list,
    'VACANCIES':vacancies_list,
    
}

#localizations = pd.read_csv('archivo.csv de localizaciones')

for job_list in job_elements:

    #ROLE NAME
    role=job_list.find('span', class_='noctitle')
    if role:
        fixed_role=role.find(string=True, recursive=False).strip() #Cuando usas Recursive, no pone los datos de los demás "span".
        role_list.append(fixed_role)
    else:
        role_list.append('Null')

    #PUBLISHED DATE
    published_date=job_list.find('li', class_='date')
    if published_date:
        fixed_date=published_date.find(string=True, recursive=False).strip() 
        dates_list.append(fixed_date)
    else:
        dates_list.append('Null')

    #BUSINESS NAME
    business_name=job_list.find(class_='business')
    if business_name:
        fixed_business=business_name.find(string=True, recursive=False).strip()
        business_list.append(fixed_business)
    else:
        business_list.append('Null')

    #LOCATION
    location=job_list.find('li', class_='location').get_text()
    if location:
        fixed_location=location[10:].strip()
        locations_list.append(fixed_location)
    else:
        locations_list.append('Null')
        
    #SALARY
    salary=job_list.find(class_='salary').get_text()
    if salary:
        fixed_salary=salary[30:].strip()
        if 'hourly' in fixed_salary:
            fix_sal=fixed_salary[:5]
        if 'anually' in fixed_salary:
            fix_sal=(fixed_salary[:9].replace(",",""))/2000
        salaries_list.append(f"$"+fix_sal)
    else:
        salaries_list.append('Null')

#ID NUMBER
for link in soup.find_all('a'): #CHANGE THIS FOR SOMETHING BETTER
    id=link.get('href')
    try:
        id.split('/')[-1:]
        id_number=id[22:30]
        try:
            id_number=int(id_number)
            id_list.append(id_number)
        except:
            pass
    except:
        pass
        
        

#print('Published date:', dates_list, '\n')
#print('Role:', role_list, '\n')
#print('Business:', business_list, '\n')
#print('Location:', locations_list, '\n')
#print('Salary:', salaries_list, '\n')
#print('ID:', id_list)

Location: ['Sexsmith (AB)', 'Calgary (AB)', 'Mississauga (ON)', 'Edmonton (AB)', 'Sarsfield (ON)', 'Scarborough (ON)', 'Gloucester (ON)', 'Newmarket (ON)', 'Calgary (AB)', 'Guelph (ON)', 'Calgary (AB)', 'Windsor (ON)', 'Brampton (ON)', 'Vittoria (ON)', 'London (ON)', 'Various locations', 'Blenheim (ON)', 'Lethbridge (AB)', 'Calgary (AB)', 'Trenton (ON)', 'Beeton (ON)', 'Edmonton (AB)', 'Bothwell (ON)', 'Welland (ON)', 'Maxville (ON)'] 



In [4]:
for ids in id_list:
    try:
        id_url='https://www.jobbank.gc.ca/jobsearch/jobposting/'+str(ids)
        id_page=requests.get(id_url)
        id_soup=BeautifulSoup(id_page.content, 'html.parser')
        job_details=id_soup.find('main', class_='container')
    
        description = job_details.find('span', property='description').get_text()
    
        #EDUCATION
        education=re.search(r'Education: ([^.]+)', description)
        if education:
            fixed_education=education.group(1)
            education_list.append(fixed_education)
        else:
            education_list.append('Null')
    
        #EXPERIENCE
        experience=re.search(r'Experience: ([^.]+)', description)
        if experience:
            fixed_experience=experience.group(1)
            experience_list.append(fixed_experience)
        else:
            experience_list.append('Null')
            
        #NOC CODE
        noccode=job_details.find('span', class_='aa_jobbank_job_noccode').get_text()
        if noccode:
            noccode_list.append(noccode)
        else:
            noccode_list.append('Null')
        
        employment_type=job_details.find('span', property='employmentType')
        if employment_type:
        #SHIFT TYPE
            fixed_employment_type=employment_type.find(string=True, Recursive=False).get_text()
            employment_type_list.append(fixed_employment_type)
    
        #JOB DURATION
            for duration in employment_type:
                job_duration=duration.get_text(strip=True)
                job_duration_list.append(job_duration)
                break
        else:
            employment_type_list.append('Null')
            job_duration_list.append('Null')
        
        #WEEKLY HOURS
        weekly_hours=job_details.find('span', property='workHours')
        if weekly_hours:
            fixed_weekly_hours=weekly_hours.get_text(strip=True)
            weekly_hours_list.append(fixed_weekly_hours)
        else:
            weekly_hours_list.append('Null')
    
        #FULL ADDRESS (WORK IN PROGRESS)
        address=job_details.find('span', property='addressLocality').get_text()
        if address:
            address_list.append(address)
        else:
            address_list.append('Null')
            
        #VACANCIES
        vacancies=job_details.find('span', string=re.compile(r'\d+ vacancies')) #Various vacancies
        if vacancies:
            vacancy_number=re.search(r'\d+', vacancies.get_text(strip=True)).group()
            vacancies_list.append(vacancy_number)
        else:
            vacancies_list.append('1')
    
        #WORK PERMIT
        work_permit=job_details.find(class_='job-posting-detail-apply').get_text()
        if "with or without a valid Canadian work permit" in work_permit:
            work_permit_list.append('No')
        else:
            work_permit_list.append('Yes')
    
    except Exception as e:
        print(f'An error occurred: {e}')
        

#print('NOC Code:', noccode_list, '\n')
#print('Employment Type:', employment_type_list, '\n')
#print('Job Duration:', job_duration_list, '\n')
#print('Weekly Hours:', weekly_hours_list, '\n')
#print('Address:', address_list, '\n')
print('Vacancies:', vacancies_list, '\n')
#print('Education:', education_list, '\n')
#print('Experience:', experience_list, '\n')
print('Work permit:', work_permit_list, '\n')

Vacancies: ['2', '2', '1', '5', '3', '1', '2', '1', '3', '3', '2', '1', '1', '5', '1', '3', '2', '4', '8', '2', '2', '5', '6', '10', '3'] 

Work permit: ['Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes'] 



In [5]:
print(jobs)

{'ID': [41486235, 41475229, 41472895, 41460166, 41447597, 41433573, 41432480, 41426072, 41423047, 41422125, 41421898, 41421273, 41420737, 41420469, 41417176, 41412591, 41404305, 41403705, 41403673, 41403240, 41402614, 41400104, 41397063, 41390813, 41385980], 'ROLES': ['driller - oil and gas drilling', 'insurance agent', 'medical administrative assistant', 'guard, security', 'farm worker, general', 'food service supervisor', 'cook', 'maintenance person - building', 'kitchen helper', 'supervisor, food services', 'cleaner', "cook's helper", 'shift manager - fast food restaurant', 'farm supervisor', 'truck dispatcher', 'delivery drivers supervisor', 'fruit farm labourer', 'long haul truck driver', 'long haul truck driver', 'cook', 'car mechanic', 'landscape worker', 'nursery worker', 'kitchen helper', 'poultry farm worker'], 'SALARY': ['$26.00', '$32.21', '$25.00', '$19.00', '$16.71', '$17.00', '$16.55', '$21.00', '$15.50', '$18.00', '$20.00', '$16.70', '$17.30', '$21.63', '$28.85', '$28.0

In [6]:
nearest_city_list=[]
distance_nearest_city_list=[]

def calc_distance(fixed_location):
    try:
        city1=re.sub(r'\s*\(AB\)', '', fixed_location)
    except:
        city1=re.sub(r'\s*\(ON\)', '', fixed_location)

    edmonton_coords=(53.5462055, -113.491241)
    red_deer_coords=(52.2690628, -113.8141464)
    calgary_coords=(51.0456064, -114.057541)
    toronto_coords=(43.6534817, -79.3839347)
    ottawa_coords=(45.4208777, -75.6901106)
    
    geolocator = Nominatim(user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0')
    #big_cities=['Edmonton','Calgary','Red Deer','Toronto','Ottawa']
    
    location_city1=geolocator.geocode(city1)
    lat_long_city1=(location_city1.latitude, location_city1.longitude)

    distance_journal={}

    distance=round((GD(lat_long_city1, edmonton_coords).km),2)
    distance_journal['Edmonton']=distance
    
    distance=round((GD(lat_long_city1, red_deer_coords).km),2)
    distance_journal['Red_deer']=distance
    
    distance=round((GD(lat_long_city1, calgary_coords).km),2)
    distance_journal['Calgary']=distance

    distance=round((GD(lat_long_city1, ottawa_coords).km),2)
    distance_journal['Ottawa']=distance

    distance=round((GD(lat_long_city1, toronto_coords).km),2)
    distance_journal['Toronto']=distance

    distance_nearest_city=min(distance_journal.values())
    nearest_city= [key for key, value in distance_journal.items() if value==distance_nearest_city] 
    #print(f'Nearest big city: ',nearest_city,'-',distance_nearest_city,' kms')

    nearest_city_list.append(nearest_city)
    distance_nearest_city_list.append(distance_nearest_city)   

In [7]:
for distances in locations_list:
    try:
        calc_distance(distances)
    except:
        nearest_city_list.append('Null')
        distance_nearest_city_list.append('Null')

print('NEAREST CITY:', nearest_city_list, '\n')
print(f'DISTANCE:', distance_nearest_city_list, '\n')

NEAREST CITY: [['Edmonton'], ['Calgary'], ['Toronto'], ['Edmonton'], ['Ottawa'], ['Toronto'], ['Ottawa'], ['Toronto'], ['Calgary'], ['Toronto'], ['Calgary'], ['Toronto'], ['Toronto'], ['Toronto'], ['Toronto'], 'Null', ['Toronto'], ['Calgary'], ['Calgary'], ['Toronto'], ['Toronto'], ['Edmonton'], ['Toronto'], ['Toronto'], ['Ottawa']] 

DISTANCE: [397.56, 0.0, 17.69, 0.0, 27.12, 16.73, 13.5, 45.19, 0.0, 70.89, 0.0, 332.89, 30.54, 125.01, 168.21, 'Null', 258.56, 173.7, 0.0, 153.65, 57.2, 0.0, 232.01, 74.28, 67.03] 



In [8]:
#read_csv(filepath_or_buffer, *[, sep, ...])

tabla=pd.DataFrame.from_dict(jobs)
#print(tabla)

tabla.to_csv('archivo.csv', header=True, index=False)
tablita=pd.read_csv('archivo.csv')
print(tablita)


          ID                                 ROLES  SALARY  \
0   41486235        driller - oil and gas drilling  $26.00   
1   41475229                       insurance agent  $32.21   
2   41472895      medical administrative assistant  $25.00   
3   41460166                       guard, security  $19.00   
4   41447597                  farm worker, general  $16.71   
5   41433573               food service supervisor  $17.00   
6   41432480                                  cook  $16.55   
7   41426072         maintenance person - building  $21.00   
8   41423047                        kitchen helper  $15.50   
9   41422125             supervisor, food services  $18.00   
10  41421898                               cleaner  $20.00   
11  41421273                         cook's helper  $16.70   
12  41420737  shift manager - fast food restaurant  $17.30   
13  41420469                       farm supervisor  $21.63   
14  41417176                      truck dispatcher  $28.85   
15  4141

In [390]:
ON_AB_localization = {
    'State': [],
    'City': [],
    'Lat': [],
    'Long': [],
}

ON_AB_localization_ONLY = pd.DataFrame.from_dict(ON_AB_localization)
ON_AB_localization_ONLY.to_csv("ON_AB_localization.csv", header=True, index=False)

In [459]:
def search_city(fixed_location):
    ON_AB_localization_EXCEL = pd.read_csv("ON_AB_localization.csv")
    location= re.sub(r'\s*\(ON\)|\s*\(AB\)', '', fixed_location)
    found = ON_AB_localization_EXCEL[ON_AB_localization_EXCEL['City'] == location]
    if not found.empty:
        lat = found['Lat'].values[0]
        long = found['Long'].values[0]
        return lat, long
    else:
        geolocator = Nominatim(user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0')
        
        if 'ON' in fixed_location:
            ON_AB_localization['State'].append('ON')
            locationX = location+", Ontario"
        elif 'AB' in fixed_location:
            ON_AB_localization['State'].append('AB')
            locationX = location + ", Alberta"
        else:
            ON_AB_localization['State'].append('??')
            
        location_city=geolocator.geocode(locationX)
        city_lat, city_long=(location_city.latitude, location_city.longitude)
        
        ON_AB_localization['City'].append(location)
        ON_AB_localization['Lat'].append(city_lat)
        ON_AB_localization['Long'].append(city_long)

        return city_lat, city_long

        ON_AB_localization_DICT = pd.DataFrame.from_dict(ON_AB_localization)
        ON_AB = pd.concat([ON_AB_localization_EXCEL, ON_AB_localization_DICT], ignore_index=True)
        ON_AB = ON_AB.drop_duplicates(subset=['City'], ignore_index=True)
        ON_AB.to_csv("ON_AB_localization.csv", header=True, index=False)
            

In [461]:
print(search_city('Vittoria (ON)'))

(42.7624226, -80.323385)


In [463]:
ON_AB_localization

{'State': ['ON', 'ON', 'ON', 'ON', 'ON', 'ON', 'ON'],
 'City': ['Vittoria',
  'Vittoria',
  'Vittoria',
  'Vittoria',
  'Vittoria',
  'Vittoria',
  'Vittoria'],
 'Lat': [42.7624226,
  42.7624226,
  42.7624226,
  42.7624226,
  42.7624226,
  42.7624226,
  42.7624226],
 'Long': [-80.323385,
  -80.323385,
  -80.323385,
  -80.323385,
  -80.323385,
  -80.323385,
  -80.323385]}

In [465]:
ON_AB

,State,City,Lat,Long
0,ON,Vittoria,42.762423,-80.323385


In [467]:
ON_AB_localization_EXCEL = pd.read_csv("ON_AB_localization.csv")
ON_AB_localization_EXCEL

,State,City,Lat,Long
